# CMU auto-graded notebook

Before you turn these assignments in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE."

---

# Building Your Own Optimizers
Optimizers tie the loss function and model parameters together by updating the model in response to the output of the loss function. In simpler terms, optimizers shape your model into its most accurate possible form by futzing with the weights. In even more simpler terms, optimizers define the way you want your model to be trained.

The selection of optimizers could have dramatic effect on the performance of your model. If you choose a complex optimizer for a simple model, you would be likely to spend much more time in training with barely improved accuracy. And if you choose a naive optimizer for a complicated model, you would probably end up with poor accuracy.  

This exercise will cover:
- *Part 1: Calling existing optimizers provided by torch.optim*
- *Part 2: Implementing and evaluating your own optimizers*


## Part 1: Calling existing optimizers provided by torch.optim
In this part, we will build up a demo pipeline, demonstrating how to involve optimizers given by `torch.optim` into the training process.

## (1a) Setting up the environment
Just run the following cell to establish the runtime environment. Note that we're using PyTorch libraries for this part of assignment.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cpu


## (1b) Building a baseline model
In this task, you will create a simple Linear Regression model with a synthetic toy dataset. Mean Squared Error (MSE) will be used as the loss metric.

Note we have set a seed for `torch.manual_seed()` for auto grading purpose. Please don't change the seed.

In [2]:
# A toy dataset of points around y = 4 * x_1 + 3 * x_2 + 2
NUM_EXAMPLES = 2000
torch.manual_seed(10605)   # DON'T change this seed for reproducibility!

X_train = torch.randn(NUM_EXAMPLES, 2)
noise = torch.randn(NUM_EXAMPLES, 1)
y_train = torch.matmul(X_train, torch.tensor([[4.], [3.]])) + 2 + noise

# Definition of the Linear Regression model
class LinearModel(nn.Module):
    def __init__(self):
        super(LinearModel, self).__init__()
        self.w = nn.Parameter(torch.tensor([[5.0, 5.0]]))  # Weight initialization
        self.b = nn.Parameter(torch.tensor(10.0))          # Bias initialization

    # Implementation of Linear Regression
    # TODO: Replace <FILL IN> with appropriate code
    def forward(self, inputs):
        """
        This function will use current parameters to predict the labels for inputs.

        Args:
            inputs (torch.Tensor): The input tensor of this Linear Regression model.

        Returns:
            torch.Tensor: Predicted label
        """
        return torch.matmul(inputs, self.w.T) + self.b


# MSE loss function
# TODO: Replace <FILL IN> with appropriate code
def loss(y_true, y_pred):
    """
    Calculate MSE loss between the predicted label and the actual label.

    Args:
        y_true (torch.Tensor): The actual label.
        y_pred (torch.Tensor): The predicted label.

    Returns:
        torch.Tensor: MSE loss value
    """
    return torch.mean((y_true - y_pred) ** 2)

In [3]:
# Test for LR_MSE
model = LinearModel()
# Sample input and output
x = torch.tensor([[1.0, 2.0]])
y = torch.tensor([3.0])

assert np.equal(model(x).item(), 25.0), "Wrong implementation of Linear Regression"
assert np.equal(loss(model(x), y).item(), 484.0), "Wrong implementation of MSE loss"

## (1c) Applying an existing optimizer
Next, we will apply an Adam optimizer provided by `torch.optim` to our previously defined Linear Regression model. You can reuse this code snippet to validate your own optimizers.


In [4]:
model = LinearModel()
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)

# Training parameters
batch_size = 32
epochs = 10

# Training loop
for epoch in range(epochs):
    for i in range(0, len(X_train), batch_size):
        # Get mini-batch data
        X_batch = X_train[i:i + batch_size]
        y_batch = y_train[i:i + batch_size]

        optimizer.zero_grad()

        # forward
        y_pred = model(X_batch)
        current_loss = loss(y_batch, y_pred)

        # backward
        current_loss.backward()

        # weight updates
        optimizer.step()

    # loss for the epoch
    print(f'Epoch {epoch + 1}/{epochs}, Loss: {current_loss.item()}')

Epoch 1/10, Loss: 8.912812232971191
Epoch 2/10, Loss: 1.2505282163619995
Epoch 3/10, Loss: 0.9521957635879517
Epoch 4/10, Loss: 0.9435355067253113
Epoch 5/10, Loss: 0.9430028200149536
Epoch 6/10, Loss: 0.9441074728965759
Epoch 7/10, Loss: 0.9469836354255676
Epoch 8/10, Loss: 0.9513348340988159
Epoch 9/10, Loss: 0.9568836688995361
Epoch 10/10, Loss: 0.9634056687355042


## Part 2: Implementing your own optimizers
In this part, you will implement four different optimizers by yourself! Don't be panic if you're not good at math stuff, we will give you enough instructions and explanations!

Generally, an optimizer class should contain three methods:
- `__init__`: Initializes required parameters.
-  `calculate_gradient`: Calculates gradients in the computational graph.
- `apply_gradient`: Updates parameters of the model.

## (2a) Gradient Descent
Let's start with the simplest Gradient Descent.

For a Linear Regression model with MSE where $\mathbf{y}=w^T\mathbf{X} + b$, the update rules of parameters at the $t$ th epoch are as follows:

$$w^{(t+1)}=w^{(t)}-\frac{\alpha}{n} \sum_{i=1}^{n} 2 \cdot (h_{w,b}(\mathbf{X}^{(i)}) - \mathbf{y}^{(i)}) \mathbf{X}^{(i)}$$

$$b^{(t+1)}=b^{(t)}-\frac{\alpha}{n} \sum_{i=1}^{n} 2 \cdot (h_{w,b}(\mathbf{X}^{(i)}) - \mathbf{y}^{(i)}))$$

where $n$ is the size of input dataset.



In [5]:
# TODO: Replace <FILL IN> with appropriate code
class GD():
    def __init__(self, lr=0.01):
        """Initializes required parameters.

        Args:
            lr (float): Learning rate.
        """
        self.lr = lr

    def calculate_gradient(self, X, y, model):
        """Calculates gradients manually for the linear regression model.

        Args:
            X (torch.Tensor): The input.
            y (torch.Tensor): The actual label.
            model: The model used to predict y with given input X.

        Returns:
            Tuple: (gradient w.r.t w, gradient w.r.t b)
        """

        n = X.shape[0]
        y_pred = model.forward(X)

        error = y_pred - y
        grads_w = (2/n) * torch.matmul(X.T, error).T
        grads_b = (2/n) * torch.sum(error)

        return grads_w, grads_b


    def apply_gradient(self, grads, model):
        """Updates parameters of the model.

        Args:
            grads (Tuple): (gradient w.r.t w, gradient w.r.t b).
            model: The model used to predict y with given input X.
        """

        grads_w, grads_b = grads
        model.w -= self.lr * grads_w
        model.b -= self.lr * grads_b

        # YOUR CODE HERE
        # raise NotImplementedError()


In [6]:
class LinearModel:
    def __init__(self):
        self.w = torch.tensor([[5.0, 5.0]])  # Initial weights
        self.b = torch.tensor(10.0)          # Initial bias

    def forward(self, X):
        return X @ self.w.T + self.b

# Evaluation
model_0 = LinearModel()
opt_0 = GD()

print("Initial loss: %.3f" % loss(model_0.forward(X_train), y_train))

epochs = 300
for i in range(epochs):
  grads = opt_0.calculate_gradient(X_train, y_train, model_0)
  opt_0.apply_gradient(grads, model_0)
  if i % 20 == 0:
    print("Loss at epoch %d: %.3f" % (i, loss(model_0.forward(X_train), y_train)))

print("Final loss: %.3f" % loss(model_0.forward(X_train), y_train))
print("w = %s, b = %s" % (model_0.w.numpy(), model_0.b.numpy()))

Initial loss: 70.262
Loss at epoch 0: 67.500
Loss at epoch 20: 30.470
Loss at epoch 40: 14.060
Loss at epoch 60: 6.780
Loss at epoch 80: 3.549
Loss at epoch 100: 2.113
Loss at epoch 120: 1.474
Loss at epoch 140: 1.190
Loss at epoch 160: 1.063
Loss at epoch 180: 1.006
Loss at epoch 200: 0.981
Loss at epoch 220: 0.970
Loss at epoch 240: 0.965
Loss at epoch 260: 0.963
Loss at epoch 280: 0.962
Final loss: 0.961
w = [[4.0558906 2.9847586]], b = 2.021142


In [7]:
# Test case
assert np.allclose(loss(model_0.forward(X_train), y_train), 0.9611, atol=1e-3), 'Wrong implementation of GD'

## (2b) Stochastic Gradient Descent
Let's take one step further. In SGD, you should randomly choose one data point to calculate the gradient each time.

In [8]:
# TODO: Replace <FILL IN> with appropriate code
class SGD():
  def __init__(self, lr=0.01):
    """Initializes required parameters.

      Args:
          lr (float): Learning rate.
    """
    self.lr = lr


  def calculate_gradient(self, X, y, model):
    """Calculates gradients in the computational graph.

      Note:
          You should try to make use of Tensorflow library to perform related calculation.

      Args:
          X (torch.Tensor): The input.
          y (torch.Tensor): The actual label.
          model: The model used to predict y with given input X.

      Returns:
          Tuple: (gradient w.r.t w, gradient w.r.t b)
    """

    # Randomly select a single data point (stochastic)
    idx = torch.randint(0, X.shape[0], (1,))
    X_sample = X[idx]
    y_sample = y[idx]

    # Get prediction for the sample
    y_pred = model.forward(X_sample)

    # Calculate the error
    error = y_pred - y_sample

    # Calculate gradients for w and b for this single point
    # The 2 comes from derivative of squared error
    grads_w = 2 * torch.matmul(error.T, X_sample)

    # b grad: 2 * (y_pred - y)
    grads_b = 2 * error.sum()

    return (grads_w, grads_b)

  def apply_gradient(self, grads, model):
    """Updates parameters of the model.

      Note:
          You should try to make use of Tensorflow library to perform related calculation.

      Args:
          grads (Tuple): (gradient w.r.t W, gradient w.r.t B).
          model: The model used to predict y with given input X.
    """
    # Extract gradients
    grads_w, grads_b = grads

    # Update parameters using gradient descent rule: param = param - lr * gradient
    model.w -= self.lr * grads_w
    model.b -= self.lr * grads_b

In [9]:
# Evaluation
model_1 = LinearModel()
opt_1 = SGD()

print("Initial loss: %.3f" % loss(model_1.forward(X_train), y_train))

epochs = 300
for i in range(epochs):
  # Calculate current gradients
  grads = opt_1.calculate_gradient(X_train, y_train, model_1)
  # Use the optimizer to update gradients
  opt_1.apply_gradient(grads, model_1)
  if i % 20 == 0:
    print("Loss at epoch %d: %.3f" % (i, loss(model_1.forward(X_train), y_train)))

print("Final loss: %.3f" % loss(model_1.forward(X_train), y_train))
print("w = %s, b = %s" % (model_1.w.numpy(), model_1.b.numpy()))

Initial loss: 70.262
Loss at epoch 0: 66.670
Loss at epoch 20: 34.439
Loss at epoch 40: 13.381
Loss at epoch 60: 7.430
Loss at epoch 80: 3.402
Loss at epoch 100: 2.068
Loss at epoch 120: 1.561
Loss at epoch 140: 1.308
Loss at epoch 160: 1.057
Loss at epoch 180: 0.976
Loss at epoch 200: 1.027
Loss at epoch 220: 1.025
Loss at epoch 240: 1.035
Loss at epoch 260: 1.052
Loss at epoch 280: 0.998
Final loss: 0.971
w = [[4.113454  2.9197123]], b = 1.9429923


In [10]:
# Test case
assert np.allclose(loss(model_1.forward(X_train), y_train), 0.9707, atol=0.05), 'Wrong implementation of SGD'

## (2c) AdaGrad

One of the shortcomings about Gradient Descent is that its performance highly depends on the initial learning rate. Poor selection of initial learning rate would lead to either slow convergence or incapability of finding local minimum. In order to solve this problem, a revised version called AdaGrad is introduced.

As its name suggests, AdaGrad makes its learning rate adaptive. To be more specific, AdaGrad uses past squared gradients to form an accumulated regularizer for its learning rate.

Assuming $g^{(t)}$ is the gradient at $t$ th epoch, AdaGrad first calculates the accumulated squared gradients:

$$r^{(t)}=r^{(t-1)}+(g^{(t)})^2$$

Then AdaGrad will use this accumulated squared gradients to regularize its learning rate:

$$w^{(t+1)}=w^{(t)}-\frac{\alpha}{\sqrt{r^{(t)}} + \epsilon} \cdot g_w^{(t)}$$
$$b^{(t+1)}=b^{(t)}-\frac{\alpha}{\sqrt{r^{(t)}} + \epsilon} \cdot g_b^{(t)}$$

where $\epsilon$ is an additive constant (usually set as $10^{-7}$) that ensures we do not divide by 0.

Note that you can reuse the `calculate_gradient` function in SGD for AdaGrad.


In [11]:
# TODO: Replace <FILL IN> with appropriate code
class AdaGrad():
  def __init__(self, lr=0.001, epsilon=1e-7):
    """Initializes required parameters.

      Note: self.accumulator is the accumulated squared gradients. It is a tuple with following format:
        (accumulator w.r.t w, accumulator w.r.t b)
        And it should be initialized with the value of 0.1

      Args:
          lr (float): Learning rate.
          epsilon (float): Additive constant that ensures we do not divide by 0.
    """
    self.lr = lr
    self.epsilon = epsilon
    # Initialize accumulators for w and b with value 0.1
    self.accumulator = (torch.tensor(0.1), torch.tensor(0.1))

  def calculate_gradient(self, X, y, model):
    """Calculates gradients in the computational graph.

      Note:
          You should try to make use of Tensorflow library to perform related calculation.

      Args:
          X (torch.Tensor): The input.
          y (torch.Tensor): The actual label.
          model: The model used to predict y with given input X.

      Returns:
          Tuple: (gradient w.r.t w, gradient w.r.t b)
    """
    # Randomly select a single data point (stochastic)
    idx = torch.randint(0, X.shape[0], (1,))
    X_sample = X[idx]
    y_sample = y[idx]

    # Get prediction for the sample
    y_pred = model.forward(X_sample)

    # Calculate the error
    error = y_pred - y_sample

    # Calculate gradients for w and b for this single point
    # The 2 comes from derivative of squared error
    grads_w = 2 * torch.matmul(error.T, X_sample)

    # b grad: 2 * (y_pred - y)
    grads_b = 2 * error.sum()

    return (grads_w, grads_b)

  def apply_gradient(self, grads, model):
    """Updates parameters of the model.

      Note:
          You should try to make use of Tensorflow library to perform related calculation.

      Args:
          grads (Tuple): (gradient w.r.t w, gradient w.r.t b).
          model: The model used to predict y with given input X.
    """
    # Extract gradients
    grads_w, grads_b = grads

    # Update accumulated squared gradients
    acc_w, acc_b = self.accumulator
    acc_w = acc_w + grads_w**2
    acc_b = acc_b + grads_b**2
    self.accumulator = (acc_w, acc_b)

    # Update parameters using AdaGrad update rule
    model.w -= self.lr * grads_w / (torch.sqrt(acc_w) + self.epsilon)
    model.b -= self.lr * grads_b / (torch.sqrt(acc_b) + self.epsilon)

In [12]:
# Evaluation
model_2 = LinearModel()
opt_2 = AdaGrad(lr=0.1)

print("Initial loss: %.3f" % loss(model_2.forward(X_train), y_train))

epochs = 300
for i in range(epochs):
  # Calculate current gradients
  grads = opt_2.calculate_gradient(X_train, y_train, model_2)
  # Use the optimizer to update gradients
  opt_2.apply_gradient(grads, model_2)
  if i % 20 == 0:
    print("Loss at epoch %d: %.3f" % (i, loss(model_2.forward(X_train), y_train)))

print("Final loss: %.3f" % loss(model_2.forward(X_train), y_train))
print("w = %s, b = %s" % (model_2.w.numpy(), model_2.b.numpy()))

Initial loss: 70.262
Loss at epoch 0: 68.047
Loss at epoch 20: 59.313
Loss at epoch 40: 54.384
Loss at epoch 60: 50.898
Loss at epoch 80: 48.182
Loss at epoch 100: 45.781
Loss at epoch 120: 43.249
Loss at epoch 140: 41.087
Loss at epoch 160: 39.465
Loss at epoch 180: 37.913
Loss at epoch 200: 36.551
Loss at epoch 220: 35.223
Loss at epoch 240: 33.439
Loss at epoch 260: 32.117
Loss at epoch 280: 30.790
Final loss: 29.542
w = [[4.658828  4.3051815]], b = 7.1320567


In [13]:
# Test case
assert np.allclose(loss(model_2.forward(X_train), y_train), 29.5416, atol=0.05), 'Wrong implementation of AdaGrad'

## (2d) Adam
Although it sounds like a powerful algorithm, AdaGrad still has a bunch of shortcomings. One of them is that as the accumulated squared gradients becoming larger, the learning rate will be pushed to 0, leading to an early stop.

Adam is designed to improve AdaGrad. It relies on adaptive moment estimation to set an independent adaptive learning rate for each parameter. The idea of Adam at the $t$ th epoch is described below:
- Calculate current gradient $g^{(t)}$
- Update biased first moment estimate
  $$m^{(t)} = \beta_1\cdot m^{(t-1)}+(1-\beta_1)\cdot g^{(t)}$$
- Update biased second raw moment estimate
  $$v^{(t)} = \beta_2\cdot v^{(t-1)}+(1-\beta_2)\cdot (g^{(t)})^2$$
- Compute bias-corrected first moment estimate
  $$\hat{m}^{(t)} = \frac{m^{(t)}}{1-\beta_1^t}$$
- Compute bias-corrected second raw moment estimate
  $$\hat{v}^{(t)} = \frac{v^{(t)}}{1-\beta_2^t}$$
- Update parameters with constrained learning rate
  $$w^{(t)} = w^{(t-1)}-\alpha\cdot\frac{\hat{m}_w^{(t)}}{(\sqrt{\hat{v}_w^{(t)}}+\epsilon)}$$
  $$b^{(t)} = b^{(t-1)}-\alpha\cdot\frac{\hat{m}_b^{(t)}}{(\sqrt{\hat{v}_b^{(t)}}+\epsilon)}$$

where $\beta_1$ and $\beta_2$ are hyperparameters for the first and the second moment estimation

As we can see from the update rule, Adam dynamically forms boundaries for its learning rate using first and second moment estimate without requiring too much memory space.

Note that you can reuse the `calculate_gradient` function in SGD for Adam.

In [14]:
# from tensorflow.tools.docs.doc_controls import T
# TODO: Replace <FILL IN> with appropriate code
class Adam():
  def __init__(self, lr=0.001, beta1=0.9, beta2=0.999, epsilon=1e-7):
    """Initializes required parameters.

      Note:
        self.m is a list of Tensor, representing first moment estimations of gradients. The initial value should be [0].
        self.v is a list of Tensor, representing second moment estimations of gradients. The initial value should be [0].
        self.t is the timestep. The initial value should be 0.

      Args:
          lr (float): Learning rate.
          epsilon (float): Additive constant that ensures we do not divide by 0.
          beta1 (float): Hyperparameter for first moment estimation
          beta2 (float): Hyperparameter for second moment estimation
    """
    self.lr = lr
    self.beta1 = beta1
    self.beta2 = beta2
    self.epsilon = epsilon
    self.m = [torch.tensor(0.0)]  # First moment estimate
    self.v = [torch.tensor(0.0)]  # Second moment estimate
    self.t = 0  # timestep counter


  def calculate_gradient(self, X, y, model):
    """Calculates gradients in the computational graph.

      Note:
          You should try to make use of Tensorflow library to perform related calculation.

      Args:
          X (torch.Tensor): The input.
          y (torch.Tensor): The actual label.
          model: The model used to predict y with given input X.

      Returns:
          Tuple: (gradient w.r.t w, gradient w.r.t b)
    """
    # Randomly select a single data point (stochastic)
    idx = torch.randint(0, X.shape[0], (1,))
    X_sample = X[idx]
    y_sample = y[idx]

    # Get prediction for the sample
    y_pred = model.forward(X_sample)

    # Calculate the error
    error = y_pred - y_sample

    # Calculate gradients for w and b for this single point
    # The 2 comes from derivative of squared error
    grads_w = 2 * torch.matmul(error.T, X_sample)

    # b grad: 2 * (y_pred - y)
    grads_b = 2 * error.sum()

    return (grads_w, grads_b)

  def apply_gradient(self, grads, model):
    """Updates parameters of the model.

      Note:
          You should try to make use of Tensorflow library to perform related calculation.

      Args:
          grads (Tuple): (gradient w.r.t w, gradient w.r.t b).
          model: The model used to predict y with given input X.
    """
    grads_w, grads_b = grads

    self.t += 1

    # If we only have one element in our lists, expand them
    if len(self.m) == 1:
        # Initialize for both w and b (m_w and m_b)
        self.m = [torch.zeros_like(grads_w), torch.zeros_like(grads_b)]
        # Initialize for both w and b (v_w and v_b)
        self.v = [torch.zeros_like(grads_w), torch.zeros_like(grads_b)]

    # Update biased first moment estimates
    self.m[0] = self.beta1 * self.m[0] + (1 - self.beta1) * grads_w
    self.m[1] = self.beta1 * self.m[1] + (1 - self.beta1) * grads_b

    # Update biased second raw moment estimates
    self.v[0] = self.beta2 * self.v[0] + (1 - self.beta2) * (grads_w ** 2)
    self.v[1] = self.beta2 * self.v[1] + (1 - self.beta2) * (grads_b ** 2)

    # Compute bias-corrected first moment estimates
    m_hat_w = self.m[0] / (1 - self.beta1 ** self.t)
    m_hat_b = self.m[1] / (1 - self.beta1 ** self.t)

    # Compute bias-corrected second raw moment estimates
    v_hat_w = self.v[0] / (1 - self.beta2 ** self.t)
    v_hat_b = self.v[1] / (1 - self.beta2 ** self.t)

    # Update parameters
    model.w -= self.lr * m_hat_w / (torch.sqrt(v_hat_w) + self.epsilon)
    model.b -= self.lr * m_hat_b / (torch.sqrt(v_hat_b) + self.epsilon)

In [15]:
# Evaluation
model_3 = LinearModel()
opt_3 = Adam(lr=0.1)
print("Initial loss: %.3f" % loss(model_3.forward(X_train), y_train))

epochs = 300
for i in range(epochs):
  # Calculate current gradients
  grads = opt_3.calculate_gradient(X_train, y_train, model_3)
  # Use the optimizer to update gradients
  opt_3.apply_gradient(grads, model_3)
  if i % 20 == 0:
    print("Loss at epoch %d: %.3f" % (i, loss(model_3.forward(X_train), y_train)))

print("Final loss: %.3f" % loss(model_3.forward(X_train), y_train))
print("w = %s, b = %s" % (model_3.w.numpy(), model_3.b.numpy()))

Initial loss: 70.262
Loss at epoch 0: 68.542
Loss at epoch 20: 39.030
Loss at epoch 40: 20.803
Loss at epoch 60: 10.552
Loss at epoch 80: 4.869
Loss at epoch 100: 2.466
Loss at epoch 120: 1.443
Loss at epoch 140: 1.065
Loss at epoch 160: 0.999
Loss at epoch 180: 1.020
Loss at epoch 200: 1.047
Loss at epoch 220: 1.106
Loss at epoch 240: 1.139
Loss at epoch 260: 0.983
Loss at epoch 280: 0.963
Final loss: 1.008
w = [[4.1860504 3.128482 ]], b = 2.0925407


In [16]:
# Test case
assert np.allclose(loss(model_3.forward(X_train), y_train), 1.0076, atol=0.05), 'Wrong implementation of Adam'
assert np.allclose(model_3.w.numpy(), [[4.1860504, 3.1284819]], atol=0.05), 'Incorrect value of model.w'